<h1 style="font-family:Impact,Arial;font-size:30px;">37005 Fundamentals of Derivative Security Pricing - Spring 2025</h1>
<h1 style="font-family:Impact,Arial;font-size:45px;">Assignment Part 3</h1>
<h2 style="font-family:Arial;">Erik Schl&ouml;gl</h2>
<p><small> School of Mathematical &amp; Physical Sciences<br>
University of Technology Sydney
</small></p>
<p>
<a href="mailto:Erik.Schlogl@uts.edu.au?Subject=37000 JIT" target="_blank">
<small><font color=MediumVioletRed>Erik.Schlogl@uts.edu.au</font></small></a>
</p>
<hr style="height:5px;border:none;color:#333;background-color:#333;" />

<h1 style="font-family:Impact,Arial;font-size:30px;">Currency option market quotes</h1>

Currency options are often quoted in the market in terms of implied volatilities, for at–the–money (ATM), and for 25% and 10% "delta" risk reversals (RR) and butterflies (BF). The quoted ATM volatility is the Black/Scholes implied volatility of an option with strike equal to the forward exchange rate. The "implied volatility" of an $x\%$ delta risk reversal is given by the difference between the Black/Scholes implied volatility of an $x\%$ delta out–of–the–money call and the Black/Scholes implied volatility of an $x\%$ delta out–of–the–money put. The "implied volatility" of an $x\%$ delta butterfly is given by the average of the Black/Scholes implied volatility of an $x\%$ delta out–of–the–money call and the Black/Scholes implied volatility of an $x\%$ delta out–of–the–money put, minus the ATM volatility. Consider the following data for currency options maturing in one month’s time (implied volatilities in percentage points):

<table>
    <tr><th>ATM </th><th> 25D RR </th><th> 25D BF </th><th> 10D RR </th><th> 10D BF </th></tr>
    <tr><th>9.5750 </th><th> -0.4500 </th><th>0.2750 </th><th>-0.7500 </th><th>1.1250 </th></tr>
</table>

The spot exchange rate is 1.47565 units of domestic currency per unit of foreign currency, the domestic interest rate $r$ is 3.285% continuously compounded and the foreign interest rate $r_f$ is 4.179% continuously compounded.

<B>Task 1:</B> Determine the Black/Scholes implied volatilities of the out–of–the–money call and put options. Plot the implied volatility smile. <I>(4 marks)</I>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -----------------------
# Given market data
# -----------------------
ATM = 0.09575      # 9.5750%
RR_25 = -0.00450   # -0.4500%
BF_25 = 0.00275    # 0.2750%
RR_10 = -0.00750   # -0.7500%
BF_10 = 0.01125    # 1.1250%

# -----------------------
# Reconstruct implied vols
# -----------------------
def compute_vols(atm, rr, bf):
    call_vol = atm + bf + 0.5 * rr
    put_vol  = atm + bf - 0.5 * rr
    return call_vol, put_vol

# 25Δ vols
vol_25C, vol_25P = compute_vols(ATM, RR_25, BF_25)
# 10Δ vols
vol_10C, vol_10P = compute_vols(ATM, RR_10, BF_10)

# Collect results
vols = {
    "10Δ Put": vol_10P,
    "25Δ Put": vol_25P,
    "ATM": ATM,
    "25Δ Call": vol_25C,
    "10Δ Call": vol_10C
}

# -----------------------
# Print results
# -----------------------
for k, v in vols.items():
    print(f"{k:8s}: {100*v:.4f}%")

# -----------------------
# Plot smile
# -----------------------
labels = list(vols.keys())
values = [vols[k] * 100 for k in labels]  # convert to %
x = np.arange(len(labels))

plt.figure(figsize=(8,5))
plt.plot(x, values, marker='o', linestyle='-', color='navy')
plt.xticks(x, labels)
plt.ylabel("Implied Volatility (%)")
plt.title("Implied Volatility Smile (from ATM, RR, BF quotes)")
plt.grid(True, linestyle="--", alpha=0.6)
plt.show()


: 

In the market quotes above, the delta determines the strikes of the out–of–the–money options, which are such that the absolute value of the option delta equals the prescribed level. For the options considered here, we have for the strike $K_{\omega,\Delta}$:
$$ K_{\omega,\Delta} = F(0,T)\exp\left\{-\omega N^{-1}(\omega e^{r_fT}\Delta)\sigma_{\omega,\Delta}\sqrt{T}+\frac12\sigma^2_{\omega,\Delta}T\right\} $$
where $F(0,T)$ is the forward exchange rate for the option maturity $T$, $\omega = 1$ for a call and $\omega = -1$ for a put, $\Delta$ is the option delta (where put option delta is negative), and $\sigma_{\omega,\Delta}$ is the Black/Scholes implied volatility of the (call or put) option with delta $\Delta$. $N^{-1}$ is the inverse of the standard normal cumulative distribution function - this is available as `norm.ppf()` in the `scipy.stats` package.

**Task 2:** Calculate the five option strikes for the European options determining the quotes in the above table. Determine the prices of European call options at these five strikes consistent with the above quotes (you may assume that put/call parity holds, i.e. the Black/Scholes implied volatilities of call and put options at a fixed strike are the same). <I>(4 marks)</I>

In [ ]:
# your code here

<h1 style="font-family:Impact,Arial;font-size:30px;">Option pricing in the presence of jumps</h1>

Consider the option pricing formula in the Black/Scholes model augmented by jumps, as derived in Lecture 10 (see Equation (70) of that lecture). Note that the relevant inputs defining the stochastic dynamics of the underlying asset under the risk-neutral measure are the initial asset price $S$, the volatility of the diffusion component $\sigma$, the continuously compounded interest rate $r$, the intensity $\lambda$ of the Poisson process triggering the jumps (see Slide 6 of the lecture), the expected proportional change $k$ in the asset price when a jump occurs (defined in Equation (24) of the lecture), and the standard deviation of the logarithmic proportional jumps $\delta$ (as defined by Equation (28) of the lecture).

**Task 3**

Using the scaffold provided, implement a Python function `JumpBlackScholes(S,K,sgm,r,T,callput,k,lam,delta,eps)`, returning the price of a European option in the Black/Scholes model augmented by jumps, with arguments of `S =` $S$, `sgm =` $\sigma$, `r =` $r$, `k =` $k$, `lam =` $\lambda$, `delta =` $\delta$, as well as the strike `K` and maturity `T` of the option, where `callput` is $1$ for a call and $-1$ for a put, and the infinite sum in Equation (70) of the lecture is truncated once further terms in the summation decrease below the threshold given by `eps`. <I>(4 marks)</I>

Note that the factorial appearing in Equation (70) can be evaluated using the function `factorial` imported from the `math` package.

**Example**

`Enter the underlying stock price: 100`\
`Enter the strike price: 110`\
`Enter the volatility: 0.2`\
`Enter continuously compounded interest rate: 0.05`\
`Enter the time to maturity: 0.5 `\
`Enter 1 for call or -1 for put option: 1 `\
`Enter the expected proportional change in the asset price when a jump occurs: 0.1 `\
`Enter the intensity of the jump process: 1.0 `\
`Enter the standard deviation of the logarithmic proportional jumps: 0.05 `\
`Enter the threshold at which to truncate the infinite sum: 1e-7 `\
`The option price is 3.6994861009744047`

In [ ]:
# your code here

stock = float(input('Enter the underlying stock price: '))
strike = float(input('Enter the strike price: '))
sigma = float(input('Enter the volatility: '))
interest = float(input('Enter continuously compounded interest rate: '))
maturity = float(input('Enter the time to maturity: '))
callput = int(input('Enter 1 for call or -1 for put option: '))
k = float(input('Enter the expected proportional change in the asset price when a jump occurs: '))
lam = float(input('Enter the intensity of the jump process: '))
delta = float(input('Enter the standard deviation of the logarithmic proportional jumps: '))
eps = float(input('Enter the threshold at which to truncate the infinite sum: '))
result = JumpBlackScholes(stock,strike,sigma,interest,maturity,callput,k,lam,delta,eps)
print('The option price is '+str(result))

**Task 4:** Consider a Black/Scholes model augmented by jumps with parameters as in the example above. If market prices of call and put options were given by such a model, what would be the (standard Black/Scholes) implied volatilities for at–the–money (ATM), and for 25% and 10% delta risk reversals and butterflies? What are the strikes of the out-of-the-money options making up the risk reversals and butterflies? Plot the implied volatility smile given by the market in this case. <I>(4 marks)</I>

In [ ]:
# your code here

**Task 5:** Note that the option pricing formula in Task 3 above has one volatility parameter and three jump parameters. Determine these parameters such that the data given in Task 1 is fitted as well as possible, in the sense of minimising the sum of the squared differences between the Black/Scholes implied volatilities given by the data and the Black/Scholes implied volatilities corresponding to the option prices from the jump/diffusion model. <I>(8 marks)</I>

In [ ]:
# your code here